In [261]:
import pandas as pd
from collections import OrderedDict
df_sport = pd.read_csv('sports_articles.csv', encoding = "ISO-8859-1")
df_vocab = pd.read_table('cor1.02.tsv', header=None)
df_ddo_vocab = pd.read_table('ddo_fullforms_2020-08-26.csv', header=None)


In [254]:



df_vocab_select_columns = df_vocab.iloc[:, [1,4]]


vocab_all_values = df_vocab_select_columns.values.ravel()
word_set = set(vocab_all_values)

df = pd.DataFrame(list(word_set), columns=["Words"])

df.sort_values(by="Words", inplace=True)

df.reset_index(drop=True, inplace=True)

ordered_dict = OrderedDict.fromkeys(word_set)

ordered_dict



OrderedDict([('smørings', None),
             ('medieanalyses', None),
             ('attributterne', None),
             ('lagunerne', None),
             ('nævnerens', None),
             ('penetreret', None),
             ('narkoman', None),
             ('efterretningen', None),
             ('deklinerede', None),
             ('titelsangen', None),
             ('skærekasse', None),
             ('negridest', None),
             ('genrejsningerne', None),
             ('dolmes', None),
             ('p-huse', None),
             ('paraguayanske', None),
             ('berørte', None),
             ('skudsalves', None),
             ('folkedansens', None),
             ('dobbelttjekkende', None),
             ('naturområderne', None),
             ('løjbænkes', None),
             ('sankede', None),
             ('tesers', None),
             ('starbåd', None),
             ('plejerne', None),
             ('arrigskabs', None),
             ('sejlbrætternes', None),
             ('

In [262]:
df_ddo_vocab

,0,1,2,3,4
0,0,0,NaN,talord,11036252
1,0.,0.,NaN,talord,11036266
2,00'er,00'er,NaN,sb. pl.,41000226
3,00'ers,00'er,NaN,sb. pl.,41000226
4,0-1-nederlag,0-1-nederlag,NaN,sb.,50000001
...,...,...,...,...,...
652515,åsynene,åsyn,NaN,sb.,12008622
652516,åsynenes,åsyn,NaN,sb.,12008622
652517,åsynet,åsyn,NaN,sb.,12008622
652518,åsynets,åsyn,NaN,sb.,12008622


In [255]:
word_to_check = "i øvrigt"

import time

start_time = time.time()

isin_dict = False

for x in range(10):
    isin_dict = word_to_check in ordered_dict

end_time = time.time()  

print(end_time)


1695847158.482367


In [258]:
import re


df_sport_text = df_sport.iloc[:, [0,1,2]]

sport_vocab = df_sport_text.values.ravel()

words_arr = []

def replace_digits(word):
    return re.sub(r'\d+', 'X', word)

def remove_specials(word):
    characters_to_remove = [':', "'", '?', ",", "."]
    new_word = word

    for char in characters_to_remove:
        new_word = new_word.replace(char, '')
    return new_word

def contains_non_alphanumeric(word):
    return bool(re.search(r'[^a-zæøåA-ZÆØÅ0-9]', word))

def formatWord(word):
    if any(char.isdigit() for char in word):
        return replace_digits(word)
    
for sentences in range(len(sport_vocab)):
    sentence = sport_vocab[sentences].strip()
    words = sentence.split()
    for word in range(len(words)):
        w = words[word]
        if any(char.isdigit() for char in w):
            w = replace_digits(w)
        if contains_non_alphanumeric(w):
            # print(w)
            w = remove_specials(w)
        # print(word)
        # result = ''.join(c for c in w if c.isalpha())
        # print(words[word])
        # print(result)
        words_arr.append(w.lower())

words_arr
words_sport_unique = set(words_arr)
# len(words_sport_unique)
words_sport_unique_list = list(words_sport_unique)
words_sport_lingo = []

# TODO :søg i alle leksikoner, søg med og uden bindestreg

for w in range(len(words_sport_unique)):
    isin_dict = words_sport_unique_list[w] in ordered_dict
    if (isin_dict == False):
        words_sport_lingo.append(words_sport_unique_list[w])


len(words_sport_lingo)


['tennistalent',
 'mundtape',
 'nahi',
 'rubiales',
 'carlos',
 'jonas',
 'cup-finalen',
 'fodboldlandshold',
 'sabalenka',
 'serginho',
 'ajax',
 'x-turneringer',
 'pinto',
 'pedersen',
 'karolina',
 'coco',
 'open-semifinalen',
 'cup-sejr',
 'sanne',
 'schweiz',
 'markussen',
 'fernando',
 'jumbo',
 'podiepladser',
 'julie',
 'vuelta-triumf',
 'magdeburg',
 'sepp',
 'stjernetrøjen',
 'elias',
 'wales',
 'fullerton',
 'fodboldpræsident',
 'x-turnering',
 'privatlektion',
 'midtjylland',
 'primoz',
 'us',
 'mikkel',
 'san',
 'toprival',
 'merrald',
 'ganna',
 'tennisdarlings',
 'emma',
 'compound',
 'temporyttere',
 'pointtab',
 'united',
 'cykelekspert',
 'aryna',
 'racerkørere',
 'holdturneringen',
 'superligamandskabet',
 'ballisager',
 'slam-vinder',
 'gylfi',
 'midtbanespiller',
 'tigst',
 'mathias',
 'landin',
 'celje',
 'max',
 'dylan',
 'tyskland',
 'vm-kys',
 'feyenoord',
 'linjeløb',
 'powerman',
 'filippo',
 'landsholdsperiode',
 'united-nedtur',
 'sendeplan',
 'sportsstjern

In [230]:



# Your input string
input_string = "10-1"

# Use regular expressions to replace numbers with a single 'X'
result_string = re.sub(r'\d+', 'X', input_string)

# Print the result
print(result_string)

X-X


In [181]:



df_sport['isResult'] = df_sport['isResult'].apply(lambda x: x.strip()) # trim values in column

results_true = df_sport.loc[df_sport["isResult"] == "True"]
results_false = df_sport.loc[df_sport["isResult"] == "False"]

assert(len(results_true) + len(results_false) == len(df_sport))


results_false

,Category,Headline,SubHeading,Link,isResult,isMaybe
1,BADMINTON,Dansk mester stopper på landsholdet med bekymr...,"Hans-Kristian Vittinghus mener, at Danmark er ...",https://www.dr.dk/sporten/badminton/dansk-mest...,False,NaN
7,KORT SPORT,Spansk landsholdsspiller lægger sag an mod fod...,Fodboldspilleren Jennifer Hermoso har lagt sag...,https://www.dr.dk/sporten/seneste-sport/spansk...,False,NaN
11,TENNIS,Selvom demonstranter afbrød tennisdarlings 'hi...,US Open-semifinalen mellem Coco Gauff og Karol...,https://www.dr.dk/sporten/tennis/selvom-demons...,False,NaN
12,VUELTA A ESPANA,"Hvis Vingegaard vil vinde Vueltaen, skal han s...",14. etape får stor betydning for danskerens ch...,https://www.dr.dk/sporten/cykling/vuelta/hvis-...,False,NaN
13,FODBOLD,KARAKTERER Mæhles målnæse bringer ham tæt på t...,"Vurdér selv herunder, hvilke karakterer landsh...",https://www.dr.dk/sporten/fodbold/karakterer-m...,False,NaN
...,...,...,...,...,...,...
118,FODBOLD,"Danske dommere sendes til Cypern, hvor bilbomb...","DBU's dommerformand fortæller, at de danske do...",https://dr.dk/sporten/fodbold/danske-dommere-s...,False,NaN
119,BADMINTON,Axelsen fik pludselig privatlektion af amerika...,Den danske badmintonspiller kan lære meget af ...,https://www.dr.dk/sporten/badminton/axelsen-fi...,False,NaN
120,FODBOLD,"'Folk forventer' dansk sejr, men landstræneren...",De danske fodboldkvinder har fået en drømmesta...,https://www.dr.dk/sporten/fodbold/folk-forvent...,False,x
121,KVINDELANDSHOLDET,Real Madrid-spiller kritiserer Danmarks tidlig...,Fodboldspilleren Sofie Svava er glad for Danma...,https://www.dr.dk/sporten/fodbold/kvindelandsh...,False,NaN


In [172]:
import pandas as pd
result_data = {'name': ['Katherine', 'James', 'Emily',
                        'Michael', 'Matthew', 'Laura'],
               'score': [98, 80, 60, 85, 49, 92],
               'age': [20, 25, 22, 24, 21, 20],
               'qualify_label': ['yes', 'yes', 'no',
                                 'yes', 'no', 'yes']}
 
# creating dataframe
df = pd.DataFrame(result_data, index=None)

df
 

,name,score,age,qualify_label
0,Katherine,98,20,yes
1,James,80,25,yes
2,Emily,60,22,no
3,Michael,85,24,yes
4,Matthew,49,21,no
5,Laura,92,20,yes
